In [1]:
from clingo.control import Control

In [2]:
def run(axioms):
    evASP = Control("0")
    evASP.add("base", [], axioms)

    evASP.ground([("base", [])])
    
    with evASP.solve(yield_=True) as handle:
        #print(handle)
        for m in handle: 
            print("Answer:\n{}\n".format(m))
            #print(m)

## FOL -Test

In [29]:
run("""

subformula(F,F) :- inputformula(F).
subformula(F,G) :- subformula(F,neg(G)).
subformula(F,G) :- subformula(F,and(G,_)).
subformula(F,G) :- subformula(F,and(_,G)).
subformula(F,G) :- subformula(F,or(G,_)).
subformula(F,G) :- subformula(F,or(_,G)).

-atom(F) :- subformula(_,F), F = neg(_).
-atom(F) :- subformula(_,F), F = and(_,_).
-atom(F) :- subformula(_,F), F = or(_,_).

atom(F) :- subformula(_,F), not -atom(F).

{in(F) : atom(F)}.
out(F) :- atom(F), not in(F).

% FO-Semantics

eval(F,1) :- subformula(_,F), atom(F), in(F).
eval(F,0) :- subformula(_,F), atom(F), not in(F).
eval(F,1) :- subformula(_,F), F=and(G,H), eval(G,1), eval(H,1).
eval(F,0) :- subformula(_,F), F=and(_,_), not eval(F,1).
eval(F,1) :- subformula(_,F), F=or(_,_), not eval(F,0).
eval(F,0) :- subformula(_,F), F=or(G,H), eval(G,0), eval(H,0).
eval(F,1) :- subformula(_,F), F=neg(G), eval(G,0). 
eval(F,0) :- subformula(_,F), F=neg(_), not eval(F,1).

end(F,Val) :- inputformula(F), eval(F,Val).


% inputformula(and(or(a,b),and(neg(a),c))).

inputformula(neg(or(a,b))).

""")

Answer:
inputformula(neg(or(a,b))) subformula(neg(or(a,b)),neg(or(a,b))) subformula(neg(or(a,b)),or(a,b)) subformula(neg(or(a,b)),b) subformula(neg(or(a,b)),a) -atom(neg(or(a,b))) -atom(or(a,b)) atom(b) atom(a) out(b) out(a) eval(b,0) eval(a,0) eval(or(a,b),0) eval(neg(or(a,b)),1) end(neg(or(a,b)),1)

Answer:
inputformula(neg(or(a,b))) subformula(neg(or(a,b)),neg(or(a,b))) subformula(neg(or(a,b)),or(a,b)) subformula(neg(or(a,b)),b) subformula(neg(or(a,b)),a) -atom(neg(or(a,b))) -atom(or(a,b)) atom(b) atom(a) in(b) out(a) eval(b,1) eval(a,0) eval(or(a,b),1) eval(neg(or(a,b)),0) end(neg(or(a,b)),0)

Answer:
inputformula(neg(or(a,b))) subformula(neg(or(a,b)),neg(or(a,b))) subformula(neg(or(a,b)),or(a,b)) subformula(neg(or(a,b)),b) subformula(neg(or(a,b)),a) -atom(neg(or(a,b))) -atom(or(a,b)) atom(b) atom(a) in(a) out(b) eval(a,1) eval(b,0) eval(or(a,b),1) eval(neg(or(a,b)),0) end(neg(or(a,b)),0)

Answer:
inputformula(neg(or(a,b))) subformula(neg(or(a,b)),neg(or(a,b))) subformula(neg(or(a,

## ADF - Semantics

In [53]:
run("""

subformula(F,F) :- ac(_,F).
subformula(F,G) :- subformula(F,neg(G)).
subformula(F,G) :- subformula(F,and(G,_)).
subformula(F,G) :- subformula(F,and(_,G)).
subformula(F,G) :- subformula(F,or(G,_)).
subformula(F,G) :- subformula(F,or(_,G)).

-atom(F) :- subformula(_,F), F = neg(_).
-atom(F) :- subformula(_,F), F = and(_,_).
-atom(F) :- subformula(_,F), F = or(_,_).
-atom(F) :- subformula(_,F), F = verum.
-atom(F) :- subformula(_,F), F = falsum.
atom(F) :- subformula(_,F), not -atom(F).

eval(F,1) :- subformula(_,F), F = verum.
eval(F,0) :- subformula(_,F), F = falsum.
eval(F,1) :- subformula(_,F), atom(F), in(F).
eval(F,0) :- subformula(_,F), atom(F), not in(F).
eval(F,1) :- subformula(_,F), F=and(G,H), eval(G,1), eval(H,1).
eval(F,0) :- subformula(_,F), F=and(_,_), not eval(F,1).
eval(F,1) :- subformula(_,F), F=or(_,_), not eval(F,0).
eval(F,0) :- subformula(_,F), F=or(G,H), eval(G,0), eval(H,0).
eval(F,1) :- subformula(_,F), F=neg(G), eval(G,0). 
eval(F,0) :- subformula(_,F), F=neg(_), not eval(F,1).

s(a). s(b). s(c). s(d).
ac(a,verum). ac(b,b).
ac(c,and(a,b)). ac(d,neg(b)).

{in(F) : s(F)}.
out(F) :- s(F), not in(F).

:- in(S), ac(S,F), eval(F,0).
:- out(S), ac(S,F), eval(F,1).

#show in/1.
#show out/1.

""")

Answer:
in(a) in(d) out(b) out(c)

Answer:
in(a) in(b) in(c) out(d)



![title](img/adf-example.PNG)